In [ ]:
import os
import math
from ttictoc import tic, toc
import csv
import seaborn as sns

print(os.getcwd())

In [ ]:
%run utils/dataset_tools.ipynb
%run utils/network_tools.ipynb
%run utils/training_tools.ipynb

In [ ]:
hzArray = np.array((0, 5, 10, 15, 20, 25, 30, 35, 40))
speedArray = np.array((0.00, 1.26363735, 1.58562983, 2.07066356, 2.571993, 3.18291372, 3.75322345, 4.33626595, 4.91413509))



In [ ]:
model_hex = MagNet()
opt_hex = torch.optim.Adam(model_hex.parameters(), lr=learning_rate)
check_hex = torch.load('SavedModels/Velocity/N2_G5_Loocv4_best.tar')
model_hex.load_state_dict(check_hex['model_state_dict'])
opt_hex.load_state_dict(check_hex['optimizer_state_dict'])
epoch_hex = check_hex['epoch']
loss_hex = check_hex['loss']

In [ ]:
model_hex.eval()
print(model_hex)

In [ ]:
geometryVal=5
geomPath='pent/'
N=2
epochs0 = 5
ll=4
testPath0 = 'data_agg/test'
loss_fn = nn.L1Loss()

training_data, testing_data, epochs, trainPath, trainLabelPath, testPath, testLabelPath = makeMagDataset(geometryVal=geometryVal, geomPath=geomPath, N=N, testPath0=testPath0, epochs0=epochs0, loocv=ll)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=1, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=1, shuffle=True)

In [ ]:
_, Z_test = test_loop(test_dataloader, model_hex, 0, 5, loss_fn, lastLoop=True, writePath=None, loocv=None)
_, Z_train = test_loop(train_dataloader, model_hex, 0, 5, loss_fn, lastLoop=True, writePath=None, loocv=None)

In [ ]:
print(Z_test.shape)
print(Z_train.shape)

In [ ]:
agg_data = np.zeros((820510, 2))

In [ ]:
agg_data[:90000,0] = Z_test[:,2]
agg_data[:90000,1] = Z_test[:,3]
agg_data[90000:720499,0] = Z_train[:,2]
agg_data[90000:720499,1] = Z_train[:,3]

for k in range(11):
    agg_data[720499+k,:] = np.array((0.0, model_hex(torch.zeros(1,3))))

In [ ]:
# Synthetic Data 
normArray = np.arange(1, 501)/200.00
# print(normArray[:10])

SynthVectors = np.random.rand(100000, 3)
idxVal = 720510
for k in range(0, 500):
    mag = normArray[k]
    for j in range(200):
        inputData = -np.sort(-SynthVectors[k:k+1,:])
        n = np.linalg.norm(inputData)
        inputData = inputData*mag/n
        v_pred = model_hex(torch.Tensor(inputData))
        
        agg_data[idxVal+200*k+j, :] = np.array((mag, v_pred))

In [ ]:
agg_data_round = agg_data
agg_data_round[:,0] = (np.round_(agg_data[:,0]*200.0)/200.0)

In [ ]:
dfRoundPent = pd.DataFrame(data=agg_data_round,
                      columns=('s_norm', 'v_pred')
                     )

In [ ]:
dfRoundPent.to_csv('PredictorDataRoundPent.csv')

In [ ]:
plt.figure(figsize=(10, 7), dpi=300)
titleFont=22
axisFont=18
legendFont=14

# palettes ['#1dd332', '#f08119', '#201dd3']
ax = sns.lineplot(x="s_norm", y="v_pred", palette=['#201dd3'], data=dfRoundPent, ci='sd')
plt.title('Velocity Prediction (m/s) vs. Sensor Reading Norm', fontsize=titleFont)
ax.set_xlabel('Norm of Sensor Reading', fontsize=axisFont)
plt.xlim([0, 2.5])
ax.set_ylabel('Predicted Velocity (m/s)', fontsize=axisFont)
plt.grid(False)
plt.savefig('VelPredictionCurvePent.png')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7), dpi=300)
plt.clf()
titleFont=22
axisFont=18
legendFont=14
ax = sns.histplot(x="s_norm", stat='probability', palette=['#201dd3'], binwidth=0.075, element='step', fill=False, data=dfRound[:720000])

plt.title('Histogram of Sensor Reading Norms', fontsize=titleFont)
ax.set_xlabel('Sensor Reading Norm', fontsize=axisFont)
plt.xlim([0, 2.5])
ax.set_ylabel('Probability Mass', fontsize=axisFont)
# ax.legend(fontsize=legendFont)
plt.grid(False)
'''
plt.plot((0.225, 0.225), (0, 0.025), 'k', (0.30, 0.30), (0, 0.025), 'k')
plt.plot((0.45, 0.45), (0, 0.025), 'r', (0.525, 0.525), (0, 0.025), 'r')
plt.plot((0.6, 0.6), (0, 0.025), 'k', (0.675, 0.675), (0, 0.025), 'k')
plt.plot((0.75, 0.75), (0, 0.025), 'r', (0.825, 0.825), (0, 0.025), 'r')
plt.plot((0.9, 0.9), (0, 0.025), 'k', (1.050, 1.050), (0, 0.025), 'k')
plt.plot((1.125, 1.125), (0, 0.025), 'r', (1.275, 1.275), (0, 0.025), 'r')
plt.plot((1.500, 1.500), (0, 0.025), 'k', (1.650, 1.650), (0, 0.025), 'k')
plt.plot((1.800, 1.800), (0, 0.025), 'r', (1.950, 1.950), (0, 0.025), 'r')
plt.plot((2.100, 2.100), (0, 0.025), 'k', (2.250, 2.250), (0, 0.025), 'k')
'''
# plt.savefig('ErrorHist.png')
plt.show()

In [ ]:
print(dfRound[:400])